In [22]:
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler
import os
from sklearn.pipeline import Pipeline
import numpy as np

In [23]:
FORESIGHT_DIRECTORY = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
DATA_INTERIM_DIRECTORY = os.path.join(FORESIGHT_DIRECTORY, "data", "interim")
DATA_PROCESSED_DIRECTORY = os.path.join(FORESIGHT_DIRECTORY, "data", "processed")

# Nombre de archivo
DATA_FILENAME = "combat_results.csv"

# Rutas completas
FILE_PATH = os.path.join(DATA_INTERIM_DIRECTORY, DATA_FILENAME)

In [24]:
df = pd.read_csv(FILE_PATH)

In [25]:
y = df['difficulty']

In [26]:
df, X_test, y_train, y_test = train_test_split(df, y, test_size=.15, random_state=42, stratify=y)

In [27]:
column_names_with_nulls = df.columns[df.isnull().any()].tolist()

# Diccionario global para guardar scalers por columna y nivel
_scalers_by_level = {}

def preprocess_df(df, is_train=True):
    df = df.copy()
    all_cols = df.columns.tolist()
    players = [all_cols[i:i+10] for i in range(0, 100, 10)]

    # Transformadores por índice de columna (excepto el índice 0)
    transformers = {
        1: lambda: MinMaxScaler(),                         # log(1 + x) + MinMax
        2: lambda: MinMaxScaler(),
        3: lambda: StandardScaler(),
        4: lambda: StandardScaler(),
        5: lambda: StandardScaler(),
        6: lambda: RobustScaler(),
        7: lambda: MinMaxScaler(),
        8: lambda: RobustScaler()
    }

    for player_cols in players:
        level_col = player_cols[1]

        for i in range(1, 9):  # del índice 1 al 8
            attr_col = player_cols[i]

            df[attr_col] = df[attr_col].astype(float)
            valid_mask = df[attr_col].notna() & df[level_col].notna()
            if not valid_mask.any():
                continue

            for level in df.loc[valid_mask, level_col].unique():
                level_mask = (df[level_col] == level) & df[attr_col].notna()
                values = df.loc[level_mask, [attr_col]].copy()

                if i == 1:  # log(1 + x) para level
                    values[attr_col] = np.log1p(values[attr_col])

                key = f"{attr_col}_lvl{level}"
                scaler = _scalers_by_level.get(key)

                if is_train:
                    scaler = transformers[i]()
                    df.loc[level_mask, attr_col] = scaler.fit_transform(values)
                    _scalers_by_level[key] = scaler
                elif scaler:
                    df.loc[level_mask, attr_col] = scaler.transform(values)
    

    # ---------- Imputación ----------
    cat_cols = [col for col in column_names_with_nulls if df[col].dtype == 'object']
    num_cols = [col for col in column_names_with_nulls if df[col].dtype != 'object']

    for col in cat_cols:
        df[col] = df[col].fillna('None')

    df[num_cols] = df[num_cols].fillna(-1)


    df["winner"] = df["winner"].map({"party": 1, "monsters": 0})
    cols_to_drop = ["party_hp_ratio"] + [df.columns[i] for i in range(0, 141, 10)]

    df = df.drop(columns=cols_to_drop)
    return df

In [28]:
df_train = preprocess_df(df, is_train=True)

In [29]:
X_test =  preprocess_df(X_test, is_train=False)

In [30]:
df_train

,pc1_level,pc1_hp_max,pc1_ac,pc1_STR,pc1_DEX,pc1_CON,pc1_INT,pc1_WIS,pc1_CHA,pc2_level,...,monster7_ac,monster7_STR,monster7_DEX,monster7_CON,monster7_INT,monster7_WIS,monster7_CHA,not_conscious_players_ratio,difficulty,num_players
379198,0.0,0.064103,-0.440123,-2.109383,-0.469217,-1.0,0.166667,1.000000,3,0.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.000000,9,3
68354,0.0,0.102564,2.210057,0.059114,1.771458,0.5,0.166667,1.000000,0,0.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.000000,9,7
379083,0.0,0.153846,1.149985,0.059114,1.024566,0.0,0.333333,0.000000,0,0.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.333333,6,3
386446,0.0,0.064103,0.089913,-1.025135,0.277675,-1.5,0.166667,0.000000,4,0.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.000000,9,5
131826,0.0,0.205128,0.619949,-1.025135,1.024566,0.0,0.666667,1.333333,4,0.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.000000,9,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
177206,0.0,0.025641,1.149985,-2.109383,1.771458,-0.5,0.500000,0.000000,3,0.0,...,17.0,3.0,0.0,2.0,1.0,0.0,2.0,1.000000,9,3
87950,0.0,0.089744,0.089913,0.059114,-0.469217,0.0,0.333333,0.666667,1,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.000000,9,1
137368,0.0,0.371795,-1.500196,1.143362,-1.216109,0.0,0.000000,0.333333,-1,0.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.000000,3,2
81341,0.0,0.153846,1.680021,0.059114,1.024566,0.5,0.333333,0.333333,2,0.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.000000,9,6


In [32]:
df_train.to_csv(os.path.join(DATA_PROCESSED_DIRECTORY, 'X_train.csv'), index=False)
X_test.to_csv(os.path.join(DATA_PROCESSED_DIRECTORY, 'X_test.csv'), index=False)